In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from data import Envelope
from system import AccountingSystem
from utils import dollars

In [3]:
sys = AccountingSystem()
sys.load()

In [4]:
import panel as pn

pn.extension("tabulator", notifications=True)
pn.widgets.Tabulator.theme = "materialize"

NotificationArea()

# Accounts Table

In [5]:
from bokeh.models.widgets.tables import BooleanFormatter, NumberFormatter, SelectEditor

In [6]:
# from panel.viewable import Viewer

# class EditableAccountsTable(Viewer):
#     def __init__(self):
#         self.setup_widgets()

#     def setup_widgets(self):
#         self.add_account_button = pn.widgets.Button(name="Add", button_type="primary")
#         self.accounts_table = pn.widgets.Tabulator(
#             sys.data.accounts,
#             formatters={
#                 "track": BooleanFormatter(),
#                 "amount": NumberFormatter(format="$0,0.00", font_style="bold"),
#             },
#             editors={
#                 "track": SelectEditor(options=["True", "False"]),
#                 "amount": {"type": "number", "verticalNavigation": "table"},
#             },
#         )
#         self.layout = pn.Column(self.accounts_table, self.add_account_button)

In [7]:
add_account_button = pn.widgets.Button(name="Add", button_type="primary")
accounts_table = pn.widgets.Tabulator(
    sys.data.accounts,
    formatters={
        "track": BooleanFormatter(),
        "amount": NumberFormatter(format="$0,0.00", font_style="bold"),
    },
    editors={
        # "track": SelectEditor(options=["True", "False"]),
        "track": None,
        "amount": {"type": "number", "verticalNavigation": "table"},
    },
    selectable=False,
)


def check_toggle_track(event):
    if event.column == "track":
        current_value = sys.data.accounts.loc[event.row, "track"]
        sys.data.accounts.loc[event.row, "track"] = not current_value
        accounts_table.value = accounts_table.value = sys.data.accounts


accounts_table.on_click(check_toggle_track)


def add_account(event):
    sys.data.accounts = sys.data.accounts.append(
        dict(name="name", amount=0.0, track=False), ignore_index=True
    )
    accounts_table.value = sys.data.accounts


add_account_button.on_click(add_account)

Watcher(inst=Button(button_type='primary', name='Add'), cls=<class 'panel.widgets.button.Button'>, fn=<function add_account at 0x7f281556f7f0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [8]:
accounts_editor = pn.Column(accounts_table, add_account_button)
accounts_editor

Column
    [0] Tabulator(editors={'track': None, ...}, formatters={'track': BooleanFormatter...}, selectable=False, value=Empty DataFrame
Columns: [...)
    [1] Button(button_type='primary', name='Add')

In [9]:
sys.data.accounts

,name,amount,track


In [10]:
sys.data.envelopes

[Envelope(id=0, name='Unaccounted', category='Internal/Unaccounted', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True),
 Envelope(id=1, name='Expense Queue', category='Internal/Expense Queue', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True),
 Envelope(id=2, name='Staged Expense', category='Internal/Staged Expense', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True)]

In [11]:
sys.take_envelopes_snapshot()
sys.take_accounts_snapshot()

In [12]:
sys.data.envelope_history

,Unaccounted,Expense Queue,Staged Expense,date
0,0.0,0.0,0.0,2022-09-08 19:22:36.518509


In [13]:
sys.data.account_history

,date
0,2022-09-08 19:22:36.522287


In [14]:
# sys.grab_accounts()
# sys.data.transfers

# Envelopes

In [15]:
import param
from panel.viewable import Viewer

envelope_colors = {
    "Cost": "#992211",
    "Emergency": "#775533",
    "Save": "#228844",
    "Spend": "#224488",
    "Internal/Unaccounted": "#449988",
    "Internal/Expense Queue": "#BB7711",
    "Internal/Staged Expense": "#AA2277",
}


class EnvelopePanel(Viewer):

    css = """
    .bk .envelope-card {
        background: #119933;
        color: white !important;
        border-radius: 20px !important;
        margin: 10px;
        line-height: .5;
    }
    
    .bk .envelope-card h1,
    .bk .envelope-card h3 {
        padding: 0px;
        margin: 0px;
    }
    .bk .envelope-card h3 {
        text-align: left;
        margin-left: -10px;
        margin-top: -5px;
        color: #FFFFFF88;
    }
    .bk .envelope-card small {
        font-size: 12px;
        color: #FFFFFFAA;
    }
    
    .bk .card-button {
        display: none !important;
    }
    
    .bk .curvy {
        border-radius: 20px !important;
    }
    
    
    .bk .arrow-toggle .bk-btn,
    .bk .arrow-toggle .bk-btn .bk-active {
        background-color: transparent;
        color: white;
        border: none;
        border-bottom: 1px solid;
        box-shadow: none;
        font-size: 20pt;
    }
    .bk .arrow-toggle .bk-btn:hover {
        color: yellow;
    }
    
    .bk .nice-select .bk-input {
        background-color: transparent;
        color: white;
        margin-right: 20px;
        padding: 0;
    }
    
    .area {
        background-color: rgba(255, 255, 255, .2);
        /*padding: 5px;*/
        border-radius: 20px;
    }
    .bk .area .nice-btn .bk-btn-group .bk-btn-default {
        background-color: rgba(255, 255, 255, .3);
        color: white;
        height: 25px;
        width: 500px;
    }
    .bk .area .nice-choices .choices .choices__inner {
        border-color: white;
    }
    .bk .area .nice-choices .choices .choices__list--dropdown  {
        border-color: white;
        color: black;
        background-color: rgba(255, 255, 255, .9);
    }
    .bk .area .nice-choices .choices .choices__list--dropdown .choices__item--selectable.is-highlighted {
        color: black;
        background-color: rgba(255, 255, 255, .85);
    }
    
    .bk .area .nice-text textarea,
    .bk .area .nice-text .bk-input {
        background-color: rgba(255, 255, 255, 0);
        color: white;
        padding: 0;
    }
    """

    def __init__(self, envelope: Envelope, system=None, global_update_callback=None):
        self.envelope = envelope
        self.global_update_callback = global_update_callback
        self.amount_pane = pn.pane.HTML("")
        
        self.system = system
        envelope_names = []
        if self.system is not None:
            envelope_names = [env.name for env in self.system.data.envelopes]
            envelope_names.remove(envelope.name)
        
        self.transfer_this_amount = pn.pane.HTML("")
        self.transfer_direction = pn.widgets.Toggle(name='>', css_classes=["arrow-toggle"], width=100, height=30)
        self.transfer_amount = pn.widgets.FloatInput(name='Amount', css_classes=["nice-select"], width=100)
        self.transfer_that_amount = pn.pane.HTML("")
        self.transfer_that_account = pn.widgets.Select(name='Envelope', options=envelope_names, css_classes=["nice-select"], width=150)
        
        self.transfer_btn = pn.widgets.Button(name="Transfer", css_classes=['nice-btn'])
        self.transfer_btn.on_click(self.transfer_clicked)
        
        self.transfer_tags = pn.widgets.MultiChoice(name='Tags', options=[], css_classes=['nice-choices'])
        self.transfer_description = pn.widgets.input.TextAreaInput(name="Description", css_classes=['nice-text'])
        
        self.transfers = pn.pane.DataFrame(None, max_rows=20, width=700)
        
        self.name_txt = pn.widgets.TextInput(name='Envelope name', value=self.envelope.name, css_classes=['nice-text'])
        self.use_goal =  pn.widgets.Checkbox(name='Use goal', value=(self.envelope.goal is not None), width=50)
        self.capped = pn.widgets.Checkbox(name='Capped', value=self.envelope.capped, width=50)
        self.goal = pn.widgets.FloatInput(name='Goal', value=(self.envelope.goal if self.envelope.goal is not None else 0.0), css_classes=['nice-select'], width=100)
        self.update_btn = pn.widgets.Button(name="Update", css_classes=['nice-btn'])
        self.delete_btn = pn.widgets.Button(name="Delete", button_type='danger')
        
        self.update_btn.on_click(self.update_clicked)

        self.layout = pn.Card(
            pn.Column(
                pn.Row(
                    pn.Column(
                        pn.Row(
                            pn.pane.HTML("<h2>Transfer</h2>"), 
                            self.transfer_amount,
                            self.transfer_that_account, 
                        ),
                        pn.Row(
                            self.transfer_this_amount,
                            self.transfer_direction,
                            self.transfer_that_amount,
                            align="center"
                        ),
                        pn.Row(self.transfer_btn, align="center"),
                    ),
                    pn.Column(
                        self.transfer_tags,
                        self.transfer_description
                    ),
                ),
                pn.Row(self.transfers, height=200, scroll=True),
                pn.Row(self.name_txt, self.use_goal, self.capped, self.goal),
                pn.Row(self.update_btn, self.delete_btn),
                # scroll=True,
                css_classes=['area'],
            ),
            header=self.collapsed_header_layout(),
            background=envelope_colors[self.envelope.category],
            header_color="white",
            header_background=envelope_colors[self.envelope.category],
            header_css_classes=["curvy"],
            # sizing_mode="stretch_both",
            css_classes=["envelope-card", "curvy"],
        )
        self.layout.collapsed = True
        self.refresh_header()
        self.update_transfers()
        super().__init__()
        
    def update_clicked(self, event):
        if not self.use_goal.value:
            self.envelope.goal = None
        else:
            self.envelope.goal = self.goal.value
        self.envelope.capped = self.capped.value
        old_name = self.envelope.name
        self.envelope.name = self.name_txt.value 
        if self.system is not None:
            self.system.data.envelope_history = self.system.data.envelope_history.rename(columns={old_name: self.envelope.name})
        # todo we also have to update the name in the original "database"? (specifically the envelope history table)
        self.global_update_callback()
        
    def update_properties(self):
        self.name_txt.value = self.envelope.name
        self.use_goal.value = (self.envelope.goal is not None)
        self.capped.value = self.envelope.capped
        self.goal.value = (self.envelope.goal if self.envelope.goal is not None else 0.0)
        
    def transfer_clicked(self, event):
        if self.system is None:
            return
        to_account = -1
        from_account = -1
        if self.transfer_direction.value:
            to_account = self.envelope
            from_account = self.system.data.get_envelope_by_name(self.transfer_that_account.value)
        else:
            to_account = self.system.data.get_envelope_by_name(self.transfer_that_account.value)
            from_account = self.envelope
        type_str = "Transfer"
        if to_account.name == "Staged Expense":
            type_str = "Expense"
        self.system.create_transfer(amount=self.transfer_amount.value, type=type_str, envelope_from=from_account.id, envelope_to=to_account.id, description=self.transfer_description.value, tags=self.transfer_tags.value)
        pn.state.notifications.success(
            f"Successfully transferred {dollars(self.transfer_amount.value)} from {from_account.name} to {to_account.name}", duration=5000
        )
        self.transfer_amount.value = 0.0
        if self.global_update_callback is not None:
            self.global_update_callback()
            
    def update_transfers(self):
        if self.system is not None:
            self.transfers.object = self.system.data.relevant_transfers(self.envelope)
            
    def refresh_transfer_account_options(self):
        envelope_names = []
        if self.system is not None:
            envelope_names = [env.name for env in self.system.data.envelopes]
            envelope_names.remove(self.envelope.name)
        self.transfer_that_account.options = envelope_names
        if len(envelope_names) > 0:
            self.transfer_that_account.value = envelope_names[0]
        
    def collapsed_header_layout(self):
        return pn.FlexBox(pn.Column(self.amount_pane))

    def expanded_header_layout(self):
        return pn.FlexBox(
            pn.Row(pn.Column(self.amount_pane), pn.Column(
                pn.pane.Str("YO DAWG")
            ))
        )

    def refresh_header(self):
        amount = f"<h1>{dollars(self.envelope.amount)}"
        if self.envelope.goal is not None:
            amount += (
                f"<small>/{dollars(self.envelope.goal, dollar_sign=False)}</small>"
            )
        amount += "</h1>"
        self.amount_pane.object = f"<h3>{self.envelope.name}</h3>{amount}"
        
        self.refresh_transfer_account_options()
        self.refresh_transfer_amounts()
        
    def refresh_transfer_amounts(self):
        self.update_transfer_values()
        if self.system is not None and self.system.data.transfer_tags is not None:
            self.transfer_tags.options = self.system.data.transfer_tags # TODO: prob needs to be moved

    @param.depends("layout.collapsed", watch=True)
    def header_clicked(self):
        if self.layout.collapsed:
            self.layout.header = self.collapsed_header_layout()
        else:
            self.layout.header = self.expanded_header_layout()
            
    @param.depends("transfer_direction.value", watch=True)
    def update_transfer_dir(self):
        if self.transfer_direction.value:
            self.transfer_direction.name = "<"
        else:
            self.transfer_direction.name = ">"
            
    @param.depends("transfer_amount.value", "transfer_that_account.value", "transfer_direction.value", watch=True)
    def update_transfer_values(self):
        this_amt = self.envelope.amount
        
        if self.system is not None:
            opposing_env = self.system.data.get_envelope_by_name(self.transfer_that_account.value)
            opposing_amt = opposing_env.amount
            if self.transfer_direction.value:
                opposing_amt -= self.transfer_amount.value
                this_amt += self.transfer_amount.value
            else:
                opposing_amt += self.transfer_amount.value
                this_amt -= self.transfer_amount.value
            self.transfer_that_amount.object = f"<p style='padding: 10px;'><b>{dollars(opposing_amt)}</b></p>"
            
        self.transfer_this_amount.object = f"<p style='padding: 10px;'><b>{dollars(this_amt)}</b></p>"
            

    def update(self):
        """Update visual layout from underlying envelope data."""
        pass

    def __panel__(self):
        return self.layout

In [16]:
pn.extension("tabulator", raw_css=[EnvelopePanel.css], notifications=True)

NotificationArea()

In [17]:
env = Envelope(name="Testing", category="Emergency", amount=123.45)

In [18]:
envelopes_example = pn.FlexBox(
    EnvelopePanel(
        Envelope(name="Cost", category="Cost", amount=123.45, goal=200.00, capped=True)
    ),
    EnvelopePanel(Envelope(name="Emergency", category="Emergency", amount=123.45)),
    EnvelopePanel(Envelope(name="Save", category="Save", amount=123.45)),
    EnvelopePanel(Envelope(name="Spend", category="Spend", amount=123.45)),
    EnvelopePanel(
        Envelope(name="Unaccounted", category="Internal/Unaccounted", amount=123.45)
    ),
    EnvelopePanel(
        Envelope(name="Expense Queue", category="Internal/Expense Queue", amount=123.45)
    ),
    EnvelopePanel(
        Envelope(
            name="Staged Expense", category="Internal/Staged Expense", amount=123.45
        )
    ),
)
envelopes_example

FlexBox(objects=[Card(background='#992211'...], sizing_mode='stretch_width')

In [19]:
# envelopes_example.show()

For normal envelopes, to do transfer:
Select from dropdown other account, then it shows: (note that the default other envelope should be Unaccounted)

    Env1 -> Env2
    amt

and the `->` is clickable to toggle transfer direction.



When click on the internal envelopes, rather than a normal envelope having related transactions etc, only have form related to that envelope:

* Unaccounted: (normal transfer), distribution plan list/button
* Expense Queue: filtered table to unaccounted expenses, so can directly handle them there 
* Staged expense: filtered table to expenses, possibly a clear all as well.

In [20]:
pn.state.notifications.position = "top-right"

def update_envs():
    sys.data.update_envelope_amounts()
    for env in envpanels:
        env.refresh_header()
        env.update_transfers()
        env.update_properties()

# accounts_editor
envpanels = []
envs_internal = []
envs_cost = []
envs_save = []
envs_emergency = []
envs_spend = []
for env in sys.data.envelopes:
    envpanel = EnvelopePanel(env, sys, global_update_callback=update_envs)
    envpanels.append(envpanel)
    if env.category.startswith("Internal"):
        envs_internal.append(envpanel)
    elif env.category == "Cost":
        envs_cost.append(envpanel)
    elif env.category == "Save":
        envs_save.append(envpanel)
    elif env.category == "Emergency":
        envs_emergency.append(envpanel)
    elif env.category == "Spend":
        envs_spend.append(envpanel)
        

pnl_envs_internal = pn.FlexBox(*envs_internal)
pnl_envs_cost = pn.FlexBox(*envs_cost)
pnl_envs_save = pn.FlexBox(*envs_save)
pnl_envs_emergency = pn.FlexBox(*envs_emergency)
pnl_envs_spend = pn.FlexBox(*envs_spend)

grab_button = pn.widgets.Button(name="Grab Accounts", button_type="default")

    
cost_add_btn = pn.widgets.Button(name="+", button_type="danger", width=30)
def add_cost(event):
    env = sys.add_envelope("New Cost", "Cost")
    envpanel = EnvelopePanel(env, sys, global_update_callback=update_envs)
    envpanels.append(envpanel)
    envs_cost.append(envpanel)
    pnl_envs_cost.objects = envs_cost
cost_add_btn.on_click(add_cost) 

save_add_btn = pn.widgets.Button(name="+", button_type="success", width=30)
def add_save(event):
    env = sys.add_envelope("New Save", "Save")
    envpanel = EnvelopePanel(env, sys, global_update_callback=update_envs)
    envpanels.append(envpanel)
    envs_save.append(envpanel)
    pnl_envs_save.objects = envs_save
save_add_btn.on_click(add_save) 

emergency_add_btn = pn.widgets.Button(name="+", button_type="warning", width=30)
def add_emergency(event):
    env = sys.add_envelope("New Emergency", "Emergency")
    envpanel = EnvelopePanel(env, sys, global_update_callback=update_envs)
    envpanels.append(envpanel)
    envs_emergency.append(envpanel)
    pnl_envs_emergency.objects = envs_emergency
emergency_add_btn.on_click(add_emergency) 

spend_add_btn = pn.widgets.Button(name="+", button_type="primary", width=30)
def add_spend(event):
    env = sys.add_envelope("New Spend", "Spend")
    envpanel = EnvelopePanel(env, sys, global_update_callback=update_envs)
    envpanels.append(envpanel)
    envs_spend.append(envpanel)
    pnl_envs_spend.objects = envs_spend
spend_add_btn.on_click(add_spend) 
    


def grab(event):
    diff = sys.grab_accounts()
    update_envs()
    if diff < 0.0:
        pn.state.notifications.warning(
            f"Negative amount ({dollars(diff, False)}) transferred to unaccounted envelope. Perhaps there are outstanding staged expenses?",
            duration=0,
        )
    elif diff == 0.0:
        pn.state.notifications.info(
            "Unaccounted envelope is up to date!", duration=5000
        )
    else:
        pn.state.notifications.success(
            f"{dollars(diff)} additional income added to unaccounted!", duration=5000
        )


grab_button.on_click(grab)



pn.FlexBox(
    accounts_editor,
    grab_button,
    pnl_envs_internal,
    pn.Row(pn.pane.HTML("<h2>Cost</h2>"), cost_add_btn),
    pnl_envs_cost,
    pn.Row(pn.pane.HTML("<h2>Save</h2>"), save_add_btn),
    pnl_envs_save,
    pn.Row(pn.pane.HTML("<h2>Emergency</h2>"), emergency_add_btn),
    pnl_envs_emergency,
    pn.Row(pn.pane.HTML("<h2>Spend</h2>"), spend_add_btn),
    pnl_envs_spend,
    
    flex_direction='column', flex_wrap='nowrap'
)
    #height=700)

FlexBox(flex_direction='column', flex_wrap='nowrap', objects=[Column
    [0] Tabulator(...], sizing_mode='stretch_height')

In [21]:
sys.data.envelopes


[Envelope(id=0, name='Unaccounted', category='Internal/Unaccounted', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True),
 Envelope(id=1, name='Expense Queue', category='Internal/Expense Queue', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True),
 Envelope(id=2, name='Staged Expense', category='Internal/Staged Expense', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True)]

Envelop layout:

* Transfer controls
* Table of related transfers
* Envelope and goal controls

In [22]:
sys.data.relevant_transfers(sys.data.envelopes[0])

,from,amount,to,type,tags,description,date


## Distribution Plan

In [31]:


# rows = []
# for envelope in sys.data.envelopes:
#     row = dict(
#         env_id=envelope.id,
#         type=envelope.category,
#         name=envelope.name,
#         goal=envelope.goal,
#         amount=None,
#         percent=None,
#         expected_increase=None
#     )
#     rows.append(row)
# table = pd.DataFrame(rows) 
# table.amount = table.amount.astype('float')
# table.percent = table.percent.astype('float')

# TODO: highlight row based on type of account.

distribution_table = pn.widgets.Tabulator(
    None,
    formatters={
        "goal": NumberFormatter(format="$0,0.00"),
        "amount": NumberFormatter(format="$0,0.00"),
        "expected_increase": NumberFormatter(format="$0,0.00"),
        "percent": NumberFormatter(format="0.0%")
    },
    editors={
        "goal": None,
        "env_id": None,
        "name": None,
        "expected_increase": None,
    },
    selectable=False,
)


dist_plan_select = pn.widgets.Select(name="Distribution plan", options=list(sys.data.distribution_plans.keys()))
new_plan_name = pn.widgets.TextInput(placeholder="New distribution plan name")
new_plan_btn = pn.widgets.Button(name="New distribution plan", button_type='primary')

sys.data.expected_income = 5_400

income_str = pn.pane.Str("Expected income:")
non_direct_str = pn.pane.Str(f"Leftover:")
remaining_pct_str = pn.pane.Str(f"Remaining distribution pct:")
unaccounted_str = pn.pane.Str(f"Unaccounted income:")


def update_strings():
    table = distribution_table.value
    if table is not None:
        leftover = sys.data.expected_income - table[~table.amount.isna()].amount.sum()

        income_str.object = f"Expected income: {dollars(sys.data.expected_income)}"
        non_direct_str.object = f"Leftover: {dollars(leftover)}"
        remaining_pct_str.object = f"Remaining distribution pct: {100 - table[~table.percent.isna()].percent.sum()*100}"
        unaccounted_str.object = f"Unaccounted income: {dollars(sys.data.expected_income - table[~table.expected_increase.isna()].expected_increase.sum())}"


def on_table_change(event):
    table = distribution_table.value
    
    leftover = sys.data.expected_income - table[~table.amount.isna()].amount.sum()
    
    
    def f(row):
        if not np.isnan(row.amount):
            return row.amount
        if not np.isnan(row.percent):
            return row.percent*leftover
        return np.NaN
    
    table.expected_increase = table.apply(f, axis=1)
    distribution_table.value = table
    sys.update_distribution_plan(dist_plan_select.value, table)
    update_strings()
    
    
    
distribution_table.on_edit(on_table_change)


def new_click(event):
    table = sys.create_distribution_plan(new_plan_name.value)
    distribution_table.value = table
    dist_plan_select.options = list(sys.data.distribution_plans.keys())
    dist_plan_select.value = new_plan_name.value
    on_table_change(event)

new_plan_btn.on_click(new_click)


def dist_plan_change(event):
    table = sys.get_distribution_plan(dist_plan_select.value)
    distribution_table.value = table
    on_table_change(event)

dist_plan_select.param.watch(dist_plan_change, "value")


pn.Column(
    dist_plan_select,
    income_str,
    non_direct_str,
    remaining_pct_str,
    unaccounted_str,
    distribution_table,
    pn.Row(new_plan_name, new_plan_btn),
)



Column
    [0] Select(name='Distribution plan', options=['test'], value='test')
    [1] Str(str)
    [2] Str(str)
    [3] Str(str)
    [4] Str(str)
    [5] Tabulator(editors={'goal': None, ...}, formatters={'goal': NumberFormatter(i...}, selectable=False)
    [6] Row
        [0] TextInput(placeholder='New distribution p...)
        [1] Button(button_type='primary', name='New distribution plan')

In [24]:
# sys.create_distribution_plan("test8")

In [25]:
# sys.data.distribution_plans

In [26]:
# sys.update_distribution_plan("test", table)

In [27]:
# sys.data.distribution_plans

In [28]:
# sys.get_distribution_plan("test8")

In [29]:
# sys.data.transfers

In [44]:
# rules:
# * If a goal is specified and the envelope is capped, the target_dist is the goal - envelope amount
# * If remaining_unaccounted == 0, *don't transfer anymore*
# 
# * If the raw amount is < target_dist and < remaining_unaccounted, *distribute the raw amount* (vacuous if above point already handled)
# * If the raw_amount is > target_dist and target_dist < remaining_unaccounted, *distribute target_dist*
# 
# * If the raw amount is < target_dist and > remaining_unaccounted, *distribute remaining_unaccounted*
# * If the raw_amount is > target_dist and target_dist > remaining_unaccounted, *distribute remaining_unaccounted*


# dist_plan_select
apply_plan_full_btn = pn.widgets.Button(name="Apply full distribution plan", button_type='primary')
apply_plan_pct_btn = pn.widgets.Button(name="Apply percent distribution plan", button_type='primary')

def apply_plan(plan_name, full=True):
    plan = sys.get_distribution_plan(plan_name)
    
    unaccounted_amt = sys.data.get_envelope_by_name("Unaccounted").amount
    
    if unaccounted_amt <= 1.0 and unaccounted_amt >= 0.0:
        pn.state.notifications.info("No unaccounted funds to distribute.")
        return
        
    
    initial_unaccounted_amt = unaccounted_amt
    dist_amounts = {}
    descriptions = {}
    
    if full:
        # =====================================
        # RAW AMOUNTS
        # =====================================
        
        # manage costs first since those are the most important
        costs = plan[(plan.type == "Cost") & (~plan.amount.isna())]
        for index, row in costs.iterrows():
            if unaccounted_amt <= 1.0:
                # no more funds! :( 
                continue
                
            env = sys.data.get_envelope_by_id(row.env_id)
            amt = row.amount
            
            # don't distribute to this env more than it wants
            if env.capped:
                target_dist = env.goal - env.amount
                
                if amt > target_dist:
                    amt = target_dist
                    
            # don't distribute more than what's left
            if amt > unaccounted_amt:
                amt = unaccounted_amt
            
            dist_amounts[row.env_id] = amt
            descriptions[row.env_id] = f"raw {dollars(row.amount)}"
            unaccounted_amt -= amt
            

        # grab any other raw amounts
        other_raw = plan[(plan.type != "Cost") & (~plan.amount.isna())]
        for index, row in other_raw.iterrows():
            if unaccounted_amt < 1.0:
                # no more funds! :( 
                continue
                
            # don't distribute more than what's left
            amt = row.amount
            if amt > unaccounted_amt:
                amt = unaccounted_amt
                
            dist_amounts[row.env_id] = amt
            descriptions[row.env_id] = f"raw {dollars(row.amount)}"
            unaccounted_amt -= amt
            
    if unaccounted_amt >= 0.0:
        # =====================================
        # PERCENT AMOUNTS
        # =====================================
        
        # divide remainder into percent buckets
        pcts = plan[~plan.percent.isna()]

        # first pass is only for envelopes that are capped
        # (figure out amounts dedicated to capped envelopes first, since this will alter the amounts transferred to other pcts)
        for index, row in pcts.iterrows():
            env = sys.data.get_envelope_by_id(row.env_id)
            if not env.capped:
                # first pass is _only_ capped envs
                continue

            target_dist = env.goal - env.amount
            if target_dist <= 0:
                # if we've already surpassed the envelope cap, skip it!
                continue

            pct_amt = row.percent * unaccounted_amt
            if pct_amt > target_dist:
                pct_amt = target_dist
            dist_amounts[row.env_id] = pct_amt
            descriptions[row.env_id] = f"pct {row.percent*100}%"
            unaccounted_amt -= pct_amt

        # second pass takes remainder and divies up completely by percent without regard to current amounts or goals
        final_unaccounted_amt = unaccounted_amt # we're not modifying unaccounted_amt at this point so as not to progressively change amount we're taking percent of
        for index, row in pcts.iterrows():
            env = sys.data.get_envelope_by_id(row.env_id)
            if env.capped:
                # we already handled capped envs!
                continue

            pct_amt = row.percent * unaccounted_amt
            dist_amounts[row.env_id] = pct_amt
            descriptions[row.env_id] = f"pct {row.percent*100}%"
            final_unaccounted_amt -= pct_amt
        
        
    # make the transfers to distribute
    full_or_pct = "full" if full else "pct"
    for key, value in dist_amounts.items():
        if value == 0.0:
            continue
        sys.create_transfer(value, type="Transfer", envelope_from=sys.data.get_envelope_by_name("Unaccounted").id, envelope_to=key, tags=[f"plan-{plan_name}", f"dist-{full_or_pct}"], description=descriptions[key])
        
    update_envs()
    
    pn.state.notifications.success(
        f"Applied distribution plan '{plan_name}', transferring {dollars(initial_unaccounted_amt-final_unaccounted_amt)} of {dollars(initial_unaccounted_amt)} ", duration=5000
    )
        
def apply_apply_plan_pct(event):
    apply_plan(dist_plan_select.value, False)
def apply_apply_plan_full(event):
    apply_plan(dist_plan_select.value, True)
apply_plan_full_btn.on_click(apply_apply_plan_full)
apply_plan_pct_btn.on_click(apply_apply_plan_pct)

pn.Column(dist_plan_select, pn.Row(apply_plan_full_btn, apply_plan_pct_btn))

Column
    [0] Select(name='Distribution plan', options=['test', 'test23'], value='test')
    [1] Row
        [0] Button(button_type='primary', name='Apply full distribution p...)
        [1] Button(button_type='primary', name='Apply percent d...)

with the quick transfer sheet, add an "populate distribution plan" so can modify applied dist plan on the fly

In [38]:
update_envs()

/home/dwl/lab/pas/data.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_df["from"] = rel_df.envelope_from.apply(lambda x:
/home/dwl/lab/pas/data.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_df["to"] = rel_df.envelope_to.apply(lambda x:
/home/dwl/lab/pas/data.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc